In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [3]:
repository = "NAS_project"

In [4]:
%cd {repository}

/content/drive/MyDrive/GitHub/NAS_project


In [ ]:
pip install nats_bench

In [ ]:
pip install yacs

In [ ]:
pip install simplejson

In [ ]:
pip install xautodl

In [ ]:
pip install pytorchcv

In [10]:
%env TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch

env: TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch


In [11]:
# import librerie

# librerie già presenti nelle prime versioni
import torch
import argparse
import datasets 
import nasspace
import pandas as pd
import csv
import random
import numpy as np
from timeit import default_timer as timer
import os
from scipy import stats
from nats_bench import create
import xautodl  # import this lib -- "https://github.com/D-X-Y/AutoDL-Projects", you can use pip install xautodl
from xautodl.models import get_cell_based_tiny_net

#librerie nuove
from pycls.models.nas.nas import Cell
import pandas as pd
from pytorchcv.model_provider import get_model as ptcv_get_model
from pytorchcv.model_provider import _models as ptcv_models
from pruners import *
from pruners import predictive
from tqdm import trange
from statistics import mean
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from GenNAS.builder_task import *
from GenNAS.builder_model import *
from GenNAS.builder_evaluator import *
from GenNAS.utils.config_generator import *


In [12]:
#SELECT DATASET
#Dataset_name = "Cifar-10"
#Dataset_name = "Cifar-100"
Dataset_name = "ImageNet"

In [14]:
GPU = '0'
seed = 1
batch_size = 128

#CIFAR10
if Dataset_name == "Cifar-10":
  dataset = 'cifar10'
  data_loc = '/content/drive/MyDrive/Github/NAS-PULITO/cifar-10-batches-py'


#CIFAR100
if Dataset_name == "Cifar-100":
  dataset = 'cifar100'
  data_loc = "/content/drive/MyDrive/NAS-PULITO/cifar-100-python"

#IMAGENET
if Dataset_name == "ImageNet":
  dataset = 'ImageNet16-120'
  data_loc = "/content/drive/MyDrive/NAS/.torch/ImageNet16"



In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
from timeit import default_timer as timer


def get_num_classes():
    return 100 if dataset == 'cifar100' else 10 if (dataset == 'cifar10') else 120


In [17]:
from timeit import default_timer as timer
import torch


def get_naswot(network, x):
  start = timer()
  network = network.to(device)

  def boolRelu_forward_hook(module, inp, out):
    if isinstance(out, tuple):
      out = out[0]
    out = out.view(out.size(0), -1)   
    x = (out > 0).float()  
    network.boolRelu = torch.cat((network.boolRelu, x), -1)
    
  network.boolRelu = torch.tensor([])
  network.boolRelu = network.boolRelu.to(device)

  for name, module in network.named_modules(): 
    if (isinstance(module, torch.nn.modules.activation.ReLU)):
      module.register_forward_hook(boolRelu_forward_hook)
              

  network(x)
  k = (network.boolRelu @ network.boolRelu.t()) + ((1. - network.boolRelu) @ (1. - network.boolRelu.t()))
  logdet = torch.linalg.slogdet(k)[1].cpu().detach()
  end = timer()
  torch.cuda.empty_cache()
  return logdet
  #return (logdet, (end - start))

In [18]:


def get_uid_and_measure(uid, input, metric_name, population = None):
  uid = int(uid)
  network = searchspace.get_network(uid)
  network.to(device)
  if metric_name == "Naswot":
    return uid, get_naswot(network, input).item()
  else:
    return 0

In [25]:
def get_uid_and_measures(uid_list, input, searchspace, metric_name):

  naswot = []

  if metric_name == "Naswot":
    for uid in uid_list:
      network = searchspace.get_network(int(uid))
      network.to(device)
      naswot.append([uid, get_naswot(network, input).item()])
    return np.array(naswot)
  else:
    return 0

In [20]:
import pandas as pd

#CIFAR 10
if Dataset_name == "Cifar-10":
  df_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar10_batch128_seed1_isomorf.csv')
  df_non_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar10_batch128_completo.csv')
  df = df_non_iso.append(df_iso)

#CIFAR100
if Dataset_name == "Cifar-100":
  df_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar100_batch128_seed1_isomorf.csv')
  df_non_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar100_batch128_completo.csv')
  df = df_non_iso.append(df_iso)

#IMAGENET
if Dataset_name == "ImageNet":
  df = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_ImageNet16-120_batch128_seed1_15625Arch.csv')


display(df)

,Unnamed: 0,arch_id,uniqArch,score,time,acc
0,0,0,(0)@avg_pool_3x3+(0)@avg_pool_3x3+(0)@nor_conv...,1281.548950,0.055238,28.211111
1,1,1,((0)@nor_conv_3x3)@avg_pool_3x3+(0)@nor_conv_3...,1380.631348,0.058419,44.488889
2,2,2,(((0)@avg_pool_3x3)@nor_conv_3x3+(0)@nor_conv_...,1263.264526,0.042976,27.633333
3,3,3,#+#+#+0,1047.412598,0.028810,29.433333
4,4,4,((0)@nor_conv_1x1+0)@nor_conv_1x1+0+0,1309.597046,0.041116,32.144444
...,...,...,...,...,...,...
15620,9155,15620,#+#+0,1054.735596,0.030721,29.822222
15621,9156,15621,#+((0)@avg_pool_3x3)@nor_conv_1x1+0,1227.618652,0.034945,31.800000
15622,9157,15622,#+((0)@nor_conv_3x3+(0)@nor_conv_3x3)@nor_conv...,1390.784668,0.049597,41.855555
15623,9158,15623,#+(#+(0)@avg_pool_3x3)@avg_pool_3x3+(0)@avg_po...,962.202087,0.030363,14.411111


In [21]:
display(df.sort_values(by=["acc"]))

,Unnamed: 0,arch_id,uniqArch,score,time,acc
13286,7642,13286,#+#+(#+#)@nor_conv_3x3,-inf,0.036226,0.833333
12746,7297,12746,#+#+#,1056.087158,0.050810,0.833333
13078,7506,13078,#+#+#,1056.087158,0.050810,0.833333
8354,4448,8354,#+#+#,1056.087158,0.050810,0.833333
632,156,632,#+#+(#+#)@avg_pool_3x3,-inf,0.038140,0.833333
...,...,...,...,...,...,...
8449,3936,8449,(#+((0)@nor_conv_3x3)@nor_conv_3x3)@nor_conv_1...,1423.089355,0.063976,46.500000
11777,5099,11777,#+(((0)@nor_conv_3x3)@nor_conv_3x3+(0)@nor_con...,1453.754639,0.063623,46.516667
3731,2003,3731,(((0)@nor_conv_3x3)@nor_conv_3x3+(0)@nor_conv_...,1447.406860,0.061208,46.555555
3888,2082,3888,(((0)@nor_conv_1x1)@nor_conv_3x3+(0)@nor_conv_...,1450.777466,0.066033,46.683333


In [22]:
df_config = pd.read_csv("conf.csv")

In [23]:
searchspace = nasspace.get_search_space(dataset, df_config.conf)
train_loader = datasets.get_data(dataset, data_loc, batch_size)

data_iterator = iter(train_loader)
input, target = next(data_iterator)
input, target = input.to(device), target.to(device)

[2022-08-28 20:30:40] Try to use the default NATS-Bench (topology) path from fast_mode=True and path=None.


# Calcolo rea con naswot

In [ ]:
from numpy.random import default_rng

Number_experiments = 3
filename = f'ReaNASWOT_on_{dataset}_in_{Number_experiments}_experimets.csv'
arch_dict = []

arch_info = ['arch_id' , 'time', 'test_acc']
history = []
from numpy.random import default_rng
population_size=20 
n_sample=5
cycles= 60
time_budget = 10 #in secondi per un singolo esperimento

for j in range(0,Number_experiments):
  print(j)

  best_arch = 0
  history = []

  start = timer()
  uid_population =  random.sample(range(len(searchspace)),population_size)
  population =  get_uid_and_measures(uid_population, input, searchspace, "Naswot") 
  history = population.copy()

  start_timer = timer()
  budget_timer = 0
  while budget_timer <= time_budget:
    control_timer = timer()
    budget_timer = -(start_timer-control_timer)
    uid_sample = random.sample(range(len(population)),n_sample)
    sample = population[uid_sample]
    parent = max(sample, key=lambda i: i[1])
    child_idx = searchspace.mutate_arch(int(parent[0]))

    if child_idx not in history[:,0]:

      uid_and_measure = np.array(get_uid_and_measure(child_idx, input, "Naswot", population))
      population = np.delete(population, 0, axis=0)
      population = np.vstack((population, uid_and_measure))
      history = np.vstack((history, uid_and_measure))

  select_from_history = history[history[:,1] == max(history[:,1])]
  print(select_from_history)
  best_arch = select_from_history[0][0]
  print(searchspace.get_final_accuracy(int(best_arch)))

  arch_dict.append({'arch_id' : int(best_arch), "time": time_budget , 'test_acc': searchspace.get_final_accuracy(int(best_arch)) })

  with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = arch_info)
    writer.writeheader()
    writer.writerows(arch_dict)






In [ ]:
ReaNASWOT_on_ImageNet16_120 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/ReaNASWOT_on_ImageNet16-120_in_30_experimets_time1.csv")
print(ReaNASWOT_on_ImageNet16_120.mean())
print(ReaNASWOT_on_ImageNet16_120.std())

arch_id     8288.266667
time           8.090000
test_acc      43.697593
dtype: float64
arch_id     4364.48443
time           0.00000
test_acc       3.01780
dtype: float64


In [ ]:
ReaNASWOT_on_ImageNet16_120 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/ReaNASWOT_on_ImageNet16-120_in_30_experimets.csv")
print(ReaNASWOT_on_ImageNet16_120.mean())
print(ReaNASWOT_on_ImageNet16_120.std())

arch_id     7742.333333
time          77.000000
test_acc      42.817593
dtype: float64
arch_id     4790.997386
time           0.000000
test_acc       3.425127
dtype: float64
